In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *

In [ ]:
## Load everything
filename = 'Source_cats/LAE_400deg_z2.5-3.2/'

pm_data = np.load(filename + 'pm_flx.npy')
pm_data_noerr = np.load(filename + 'pm_flx_no_line_no_err.npy')

mock = {}

aux = []
with open(filename + 'SEDs.csv') as csv_file:
    rdr = csv.reader(csv_file, delimiter=',')
    for row in rdr:
        aux.append(row)
mock['SEDs'] = np.array(aux, dtype=float)
aux = []
with open(filename + 'SEDs_no_line.csv') as csv_file:
    rdr = csv.reader(csv_file, delimiter=',')
    for row in rdr:
        aux.append(row)
mock['SEDs_no_line'] = np.array(aux, dtype=float)

utils = np.load(filename + 'utils.npy', allow_pickle=True).item()

mock['redshift_Lya_Arr'] = utils['z_Arr']
mock['w_Arr'] = utils['w_Arr']
mock['EW_Arr'] = utils['EW_Arr']

## Somehow, some bright mocks are sneaking in. So, get rid of them.
mask_bright = np.where(pm_data_noerr[-3] <= 1e-17)[0]
pm_data_noerr = pm_data_noerr[:, mask_bright]
pm_data = pm_data[:, mask_bright]
for key in mock.keys():
    if key == 'w_Arr': continue
    mock[key] = mock[key][mask_bright]

err_fit_params = np.load('npy/err_fit_params_minijpas.npy')
m = err_fit_params[:, 0].reshape(-1, 1)
b = err_fit_params[:, 1].reshape(-1, 1)
pm_err = np.load(filename + 'pm_flx_err.npy')[:, mask_bright]

w_central = central_wavelength()

bb_fwhm = [
    nb_fwhm(-4),
    nb_fwhm(-3),
    nb_fwhm(-2),
    nb_fwhm(-1)
]

In [ ]:
N_sources = pm_data.shape[1]

In [ ]:
sel, sel_single, list_lya, _ = QSO_find_lines(
    pm_data, pm_err, 6, 50, 20, 15, 6
)

In [ ]:
nice_lya_Arr_bool = np.zeros(N_sources).astype(bool)
nice_lya_Arr_bool[sel_single] = True

zspec = mock['redshift_Lya_Arr']
z_nb_Arr = np.ones(N_sources) * -1
for src in sel_single:
    z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1

fig, ax = plt.subplots(figsize=(8, 6))
bins = np.linspace(2, 5, 20)
#ax.hist(zspec, bins=bins, label='All', histtype='step')
ax.hist(z_nb_Arr[nice_lya_Arr_bool], bins=bins,
    alpha=0.4, color='C0')
ax.hist(z_nb_Arr[nice_lya_Arr_bool & (np.abs(zspec - z_nb_Arr) < 0.1)], bins=bins,
    label='Correct redshift', color='C0')
ax.set_ylabel('N', fontsize=15)
ax.set_xlabel(r'Computed $z_{\mathrm{Ly}\alpha}$', fontsize=15)
ax.legend(fontsize=15)
ax.set_title('Multiple QSO lines', fontsize=20)
ax.set_xlim((2, 4.5))
plt.show()

## Completeness
fig, ax = plt.subplots(figsize=(8, 6))

bins = np.linspace(2.55, 4, 20)
h_zspec, b = np.histogram(zspec[zspec > 2.5], bins=bins)
h_detec, _ = np.histogram(zspec[nice_lya_Arr_bool], bins=bins)
h_detec_r, _ = np.histogram(zspec[nice_lya_Arr_bool & (np.abs(zspec - z_nb_Arr) < 0.1)],
    bins=bins)

bin_centers = [(b[i + 1] + b[i]) / 2 for i in range(len(b) - 1)]
ax.scatter(bin_centers, h_detec_r / h_zspec, label='Completeness')
ax.scatter(bin_centers, h_detec_r / h_detec, label='Correct z')
ax.set_xlabel(r'Computed $z_{\mathrm{Ly}\alpha}$', fontsize=15)
ax.set_ylabel('N / N_sources', fontsize=15)
ax.legend(fontsize=15, loc=6)
ax.set_title('Multiple QSO lines', fontsize=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
bins = 30
#ax.hist(zspec, bins=bins, label='All', histtype='step')
ax.hist(flux_to_mag(pm_data[-2, np.where(nice_lya_Arr_bool)[0]], w_central[-2]),
    bins=bins,
    alpha=0.4, color='C0')
ax.hist(flux_to_mag(pm_data[-2, np.where(nice_lya_Arr_bool & (np.abs(zspec - z_nb_Arr) < 0.1))[0]],
    w_central[-2]), bins=bins,
    label='Correct redshift', color='C0')
ax.set_ylabel('N', fontsize=15)
ax.set_xlabel('rSDSS', fontsize=15)
ax.legend(fontsize=15)
ax.set_title('Multiple QSO lines', fontsize=20)
plt.show()

## Completeness
fig, ax = plt.subplots(figsize=(8, 6))

bins = np.linspace(21.2, 25, 20)
h_r, b = np.histogram(flux_to_mag(pm_data[-2, zspec > 2.5],
    w_central[-2]), bins=bins)
h_detec, _ = np.histogram(flux_to_mag(
    pm_data[-2, nice_lya_Arr_bool], w_central[-2]),
    bins=bins)
h_detec_r, _ = np.histogram(flux_to_mag(
    pm_data[-2, nice_lya_Arr_bool & (np.abs(zspec - z_nb_Arr) < 0.1)], w_central[-2]),
    bins=bins)

bin_centers = [(b[i + 1] + b[i]) / 2 for i in range(len(b) - 1)]
ax.scatter(bin_centers, h_detec_r / h_r, label='Completeness')
ax.scatter(bin_centers, h_detec_r / h_detec, label='Correct z')
ax.set_xlabel('rSDSS', fontsize=15)
ax.set_ylabel('N / N_sources', fontsize=15)
ax.legend(fontsize=15)
ax.set_title('Multiple QSO lines', fontsize=20)
plt.show()